In [1]:
import advent

maze = advent.get_char_grid(15).astype(int)

In [21]:
def cost(node, maze): # Safe-accessor
    return maze[node[0], node[1]] if walkable(node, maze) else 999999

def walkable(node, maze):
    return (0 <= node[0] < maze.shape[0]) and (0 <= node[1] < maze.shape[1])

def adjacent(node, maze):
    # Returns list like [(adj_i, g_i)] where g_i is cost of going from node to adj_i
    x, y = node
    return [((x+a, y+b), cost((x+a, y+b), maze)) for a in range(-1, 2) for b in range(-1, 2) if (a != 0) != (b != 0)]

grid_size = (1, 1)

In [2]:
def mod9(n):
    return 9 if n % 9 == 0 else n % 9

def cost(node, maze): # Safe-accessor
    return mod9(maze[node[0] % maze.shape[0], node[1] % maze.shape[1]] + node[0] // 10 + node[1] // 10) if walkable(node, maze) else 999999

def walkable(node, maze):
    return (0 <= node[0] < 5*maze.shape[0]) and (0 <= node[1] < 5*maze.shape[1])

def adjacent(node, maze):
    # Returns list like [(adj_i, g_i)] where g_i is cost of going from node to adj_i
    x, y = node
    return [((x+a, y+b), cost((x+a, y+b), maze)) for a in range(-1, 2) for b in range(-1, 2) if (a != 0) != (b != 0)]

grid_size = (5, 5)

In [3]:
# Copied from 2016, 13.
# Todo potentially move this to a library if possible
from tqdm.notebook import tqdm

start = (0, 0)

grid_size = grid_size  # (1, 1) for part 1. (5, 5) for part 2
target = (grid_size[0]*maze.shape[0] - 1, grid_size[1]*maze.shape[1] - 1)

# Use manhattan since cost is always >= 1
h = lambda node: abs(node[0] - target[0]) + abs(node[1] - target[1])

open = [start]
closed = []
open_parents, closed_parents = [None], []
f = [(0, h(start))] # (g, h)

log_record = 0
progress=0  # out o


with tqdm(total=1) as pbar:

    while len(open) > 0:
        current_ix = f.index(min(f, key=sum))
        current_node, current_f = open[current_ix], f.pop(current_ix)
        closed.append(open.pop(current_ix))
        closed_parents.append(open_parents.pop(current_ix))

        if current_node == target:
            print(f"Final path length: {sum(current_f)}")
            break
        #if (current_node[0] + current_node[1]) > (log_record + 10):
        #    print("Considering:", len(open), "Current:", current_node, "Target:", target)
        #    log_record = current_node[0] + current_node[1]

        # Here are our two day-specific functions, edit their signature if neccecary
        for adj, adj_g in adjacent(current_node, maze):
            if not walkable(adj, maze) or adj in closed:
                continue
            if adj not in open:
                new_f = (current_f[0] + adj_g, h(adj))
                open.append(adj)
                f.append(new_f)
                open_parents.append(current_node)
            else:
                ix = open.index(adj)
                new_f = (current_f[0] + adj_g, h(adj))
                if sum(new_f) < sum(f[ix]):
                    open_parents[ix] = current_node
                    f[ix] = new_f
        
        # Update progress bar
        progress_tmp = len(closed) / ((target[0]+1) * (target[1]+1))
        if progress_tmp > progress:
            pbar.update(progress_tmp - progress)
            progress = progress_tmp

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 2815
